In [12]:
# Imports, as always...
import numpy as np
import matplotlib.pyplot as plt 
import pandas as pd

# For generating prime numbers.
from Crypto.Util import number

# Notebook progress bars.
from tqdm.notebook import tqdm

from sqif import CVP, solve_cvp

In [10]:
def give_me_a_blank_results_dataframe():
    return pd.DataFrame(
        {
            'Bit-length' : [],
            'Lattice Dimension' : [],
            'N' : [],
            'l' : [],
            'c' : [],
            '|b_op - t|^2' : [],
            'P(b_op)' : [],
            '|v_best - t|^2' : [],
            'P(v_best)' : [],
            'P(|v_new - t|^2 < |b_op - t|^2)' : [],
            'P(|v_new - t|^2 > |b_op - t|^2)' : [],
            'E[|v_new - t|^2]' : []
        }
    )

# VQAs for CVP

A very important part of Yan et al. (2022)'s proposal for a quantum-accelerated variant of Schnorr's classical factoring algorithm is that an approximate solution to the closest vector problem (CVP) could be improved upon by considering a superposition over the states forming the unit hypercube around that solution, then using a variational algorithm (they use QAOA) to sample higher quality solutions from that search space.

The relevance of this within the picture of factoring is that we have, by Schnorr's classical factoring method, a sieve-based factoring method requiring sufficiently many "smooth-relation pairs (sr-pairs)" which we may found by reducing the problem to a closest vector problem (CVP) on the prime lattice whose elemental points map to sr-pairs. The higher the quality of our solution to the CVP, the 'better' our sr-pairs are more likely to be, and so we require fewer of them to form a suitable system of equations whose solution yields (part of) a solution to the original factorisation. 

Of course, Schnorr's algorithm is deeply flawed, but that is besides the point -- we know in what ways it is flawed, and why, thanks to works such as Léo Ducas' excellent repository. Our concern, in this notebook, is that *throwing the thing into a variational algorithm is not the silver bullet*; we suggest that Yan et al. (2022)'s proposal is built on a weak claim about lattice dimension, and so we do not expect better results even when using a quantum algorithm to improve the solution to the CVP. Indeed, we doubt their methodology is sophisticated enough to even improve the CVP effectively as the lattice dimension increases, particularly as the lattice dimension is given to scale "sublinearly" based on claims that are not expected to hold.

So, in summary, the purpose of this notebook is to empirically demonstrate that their variational approach, with a Hamiltonian mapped as they describe, offers an insufficient scaling with respect to the probability to sample better solutions to the CVP and lattice dimension, and thus their approach would not scale well even when issues with Schnorr's classical approach are not fundamental (which, unfortunately, they are).

**Hopefully, we can also try out a few other quantum heuristics (e.g. VQE, AQC-PQC), and maybe even a brute-force search of the entire hyper-cube to demonstrate that the idea of 'searching around the approximate solution on the off chance a rounding operation was not so good' is wholy insufficient to solve a problem as difficult as prime factorisation.**

In [2]:
# The expected quality of the solution.
def expected_dist(solutions, probabilities, t):
    # Compute each solution's distance to the target.
    compute_dist = lambda x : np.linalg.norm(x - t)
    dists = np.apply_along_axis(compute_dist, 1, solutions)
    
    # Take a weighted average over these distances (weighted by their probabilities).
    return np.average(dists, weights=probabilities)

In [3]:
# The probability to measure the best state...
def report_algorithm_effectiveness(solutions, probabilities, approximate_solution, t):
    approximate_quality = np.linalg.norm(approximate_solution - t)
    print(f'Our approximate solution by Babai\'s algorithm has a distance to t of {round(approximate_quality, 3):.3f}.\n')
    
    # First, determine which is the best state! This computation will scale exponentially with problem size, so let's not do this in practice -- just for analysis every once in a while.
    best_dist_to_target = np.inf
    best_prob = 0
    for v_new, prob in zip(solutions, probabilities):
        dist_to_target = np.linalg.norm(v_new - t)
        
        if dist_to_target < best_dist_to_target:
            best_dist_to_target = dist_to_target
            best_prob = prob
            
    print(f'The closest we get to t is {round(best_dist_to_target, 3):.3f}, which we obtain with a probability {round(best_prob, 3):.3f}.\n')
    
    # The cumulative probability to measure a state corresponding to a solution that is a least as good as b_op...
    better_cum_prob = 0
    worse_cum_prob = 0
    for v_new, prob in zip(solutions, probabilities):
        dist_to_target = np.linalg.norm(v_new - t)
        
        if dist_to_target < approximate_quality:
            better_cum_prob += prob
        elif dist_to_target > approximate_quality:
            worse_cum_prob += prob
            
    print(f'The probability to obtain a state vector corresponding to a solution that is BETTER than the classically-obtained approximate solution is {round(better_cum_prob, 3):.3f}.')
    print(f'The probability to obtain a state vector corresponding to a solution that is WORSE than the classically-obtained approximate solution is {round(worse_cum_prob, 3):.3f}.\n')
    
    print(f'The EXPECTED distance to t is {expected_dist(solutions, probabilities, cvp.t):.3f}')

In [35]:
# The integer bit-length we'd like to solve for.
n_bits = 48

# Lattice and precision parameters.
l = 1
c = 4

# Generating a prime number for which the above lattice dimension is required.
N = number.getPrime(n_bits)
#N=1961
print(f'Integer to factor: {N}')

# Set up the CVP.
cvp = CVP()
cvp.generate_cvp(N, l=l, c=c, seed=42)
print(f'Lattice dimension: {cvp.m}')

print('\n...')

# Solve it!
solutions, probabilities, approximate_solution = solve_cvp(cvp, n_samples=1000, delta=.75, p=1, min_method='Nelder-Mead', verbose=False)
report_algorithm_effectiveness(solutions, probabilities, approximate_solution, cvp.t)

Integer to factor: 169183578713153
Lattice dimension: 8

...
Our approximate solution by Babai's algorithm has a distance to t of 13.115.

The closest we get to t is 13.115, which we obtain with a probability 0.022.

The probability to obtain a state vector corresponding to a solution that is BETTER than the classically-obtained approximate solution is 0.000.
The probability to obtain a state vector corresponding to a solution that is WORSE than the classically-obtained approximate solution is 0.978.

The EXPECTED distance to t is 16.082


## Running Experiments

In [4]:
# From-scratch QAOA-based "SQIF" algorithm on the generated CVP.
def from_scratch_qaoa_experiment(n_bits, l, c, seed=42, n_samples=1000, delta=.75, p=1):
    N = number.getPrime(n_bits)
    
    # Set up the CVP.
    cvp = CVP()
    cvp.generate_cvp(N, l=l, c=c, seed=seed)
    
    # Solve it!
    solutions, probabilities, approximate_solution = solve_cvp(cvp, n_samples=n_samples, delta=delta, p=p, min_method='Nelder-Mead', verbose=False)
    
    approximate_quality = np.linalg.norm(approximate_solution - cvp.t)
    
    # Determine the quality of the SQIF algorithm's solutions to the CVP (in terms of probability).
    
    # First, determine which is the best state! This computation will scale exponentially with problem size, so let's not do this in practice -- just for analysis every once in a while.
    best_dist_to_target = np.inf
    best_prob = 0
    for v_new, prob in zip(solutions, probabilities):
        dist_to_target = np.linalg.norm(v_new - cvp.t)
        
        if dist_to_target < best_dist_to_target:
            best_dist_to_target = dist_to_target
            best_prob = prob
    
    # The cumulative probability to measure a state corresponding to a solution that is a least as good as b_op...
    better_cum_prob = 0
    worse_cum_prob = 0
    for v_new, prob in zip(solutions, probabilities):
        dist_to_target = np.linalg.norm(v_new - cvp.t)
        
        if dist_to_target < approximate_quality:
            better_cum_prob += prob
        elif dist_to_target > approximate_quality:
            worse_cum_prob += prob
            
    return pd.DataFrame(
        {
            'Bit-length' : [N.bit_length()],
            'Lattice Dimension' : [cvp.m],
            'N' : [N],
            'l' : [l],
            'c' : [c],
            '|b_op - t|^2' : [approximate_quality],
            'P(b_op)' : [1 - (better_cum_prob + worse_cum_prob)],
            '|v_best - t|^2' : [best_dist_to_target],
            'P(v_best)' : [best_prob],
            'P(|v_new - t|^2 < |b_op - t|^2)' : [better_cum_prob],
            'P(|v_new - t|^2 > |b_op - t|^2)' : [worse_cum_prob],
            'E[|v_new - t|^2]' : [expected_dist(solutions, probabilities, cvp.t)]
        }
    )

from_scratch_qaoa_experiment(11, l=1, c=1.5)

,Bit-length,Lattice Dimension,N,l,c,|b_op - t|^2,P(b_op),|v_best - t|^2,P(v_best),P(|v_new - t|^2 < |b_op - t|^2),P(|v_new - t|^2 > |b_op - t|^2),E[|v_new - t|^2]
0,11,3,1471,1,1.5,5.477226,0.491,5.477226,0.491,0,0.509,5.828433


### QAOA-based, Independent-training

The SQIF algorithm using a QAOA-based quantum "speed-up" mechanism, independently training and running in each input semi-prime. This is taxing on resources, and only really gives us useful insight into the algorithm as applied as a framework requiring a full training setup for every input, but it is necessary to understand a broad range of input independently under the same framework.

In [14]:
# For reference, with n_bits in [5, 50] and 5 repeats for each, p=1 takes ~5 min, p=2 takes ~30 min, p=3 takes ~75 min.

# Number of QAOA layers.
for p in tqdm(range(1, 4)):
    results = give_me_a_blank_results_dataframe()
    
    # Bit-length for the input semi-prime.
    for n_bits in range(5, 51):
        # How many repeats for each bit-length.
        for _ in range(5):
            experiment_outcome = from_scratch_qaoa_experiment(n_bits, l=1, c=1.5, seed=42, p=p)
            results = pd.concat([results, experiment_outcome])
            
    results.to_csv(f'./results/quantum-accelerated-cvp/qaoa-based-(p={p})-independent-training.csv')

  0%|          | 0/3 [00:00<?, ?it/s]

## Plotting Results

### QAOA-based, Independent-training